## Extracting location

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
import json

database_credential = json.load(open('credentials.json'))
connection = create_engine(database_credential['db'])

In [ ]:
response = requests.get('https://madchef.com.bd/contact#branches')
print("The status code is ", response.status_code)
scraped_data = BeautifulSoup(response.text, 'html.parser')
locations = scraped_data.find_all("div", {"class": "branch"})

In [ ]:
location_list = []

for i in range(len(locations)):
    location = locations[i].select_one('.branch-name').text.replace("\r\n","").strip()
    location = location.replace("(Delivery)", "").strip()
    location_list.append(location)

In [ ]:
location_data = pd.DataFrame({'location': location_list})
location_data['restaurant'] = None
location_data['restaurant'] = "madchef"

location_data.to_sql(con=connection, name = 'location_data', if_exists='append', index=False)

## Extracting Product name and price

In [14]:
response = requests.get('https://madchef.com.bd/menu')
print("The status code is ", response.status_code)
scraped_data = BeautifulSoup(response.text, 'html.parser')

The status code is  200


In [15]:
product_name = scraped_data.find_all("div", {"class": "menu-item-title"})
product_price = scraped_data.find_all("div", {"class": "menu-item-price"})

In [16]:
name_list = []
price_list = []

for i in range(len(product_name)):
    name = product_name[i].text.replace("\n ","").strip()
    name = name.replace("🌶", "").strip()
    name_list.append(name)
    
    price = product_price[i].text.replace("\n", "")
    price = price.replace("৳","").strip()
    price_list.append(price)
    
prod_price = pd.DataFrame({'name': name_list, 'price': price_list} )
prod_price['restaurant'] = None
prod_price['restaurant'] = 'madchef'

In [17]:
prod_price.head(5)

,name,price,restaurant
0,Chicken Cheese Bombs,299,madchef
1,Spicy Cajun Fries,159,madchef
2,Fries - Salt & Pepper,139,madchef
3,Onion Rings,149,madchef
4,Spicy Garlic Mushrooms,269,madchef


In [19]:
prod_price.to_csv("./data/1_madchef.csv", index=False)
prod_price.to_sql(con=connection, name = 'madchef', if_exists='append', index=False)